# About

This notebook does the following things at a high level:
1. code for generating adversarial PGD images and saving it to be used for fine-tuning
2. some visualizations of the perturbed images
3. creating the dataset (with some random augmentations)
4. finetuning the robust model  
5. evaluating on test set 

# 0. Importing required libraries


# 1. Constants used 

# 2. Generating & Saving Perturbed Images in the dataset 

# 3. Creating the fine-tuning dataset

# 4. Implementing Robust Resnet

# 5. Saving model & Evaluating Results